In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import torch
import transformers
import shap
import pickle
np.random.seed(0)

def select_df(df, lang='en', withheld='DE', exclude_fully_blocked_users=False, only_include_fully_blocked_users=False, exclude_certain_retweet_targets=None):
    if withheld is not None:
        new_df = df[df['withheld'].fillna('').str.contains(f"'{withheld}'")]
    else:
        new_df = df
    new_df = new_df.loc[new_df['lang'] == lang]

    if exclude_fully_blocked_users:
        import pandas as pd
        user_df = pd.read_csv('/home/public/twitter_politics/censorship/data/paper_data/users.csv')
        new_df = new_df[~new_df['author_id'].isin(user_df['id'].values.tolist())]
        return new_df
    
    if only_include_fully_blocked_users:
        # an author_id list
        new_df = new_df[new_df['author_id'].isin(only_include_fully_blocked_users)]

        return new_df
    
    if exclude_certain_retweet_targets:
        for exclude_target in exclude_certain_retweet_targets:
            new_df = new_df[~new_df['text'].str.contains(f"RT @{exclude_target}")]

    return new_df
out_str = '/nfs/turbo/coe-vvh/ljr/Censorship/data/dataset_test_p.csv'
data = pd.read_csv(out_str, engine='python', index_col=0)
data_out = select_df(data, lang='en', withheld=None)
data_out.to_csv(out_str[:-4] + '_en.csv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
!python3 train.py --dataset_df_dir data/ --splits_filename dataset_train_p_en.csv dataset_val_p_en.csv dataset_test_p_en.csv --text_col full_text_proc --y_col censored --class_weight automatic --seed 42 --model_save_dir models/ --log_dir log/ --iter_time_span 1000 --csv_output_path output/roberta-base-proc-new-p-en.csv --pretrained_model roberta-base --n_epochs 1 --output_type binary --lr 1e-5


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.